In [1]:
import numpy as np
import pandas as pd
#import statsmodels.api as sm
import databricks.koalas as ks
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import time
#import datetime


In [6]:
# instalar java jdk ---> sudo apt-get install -y openjdk-8-jdk-headless 
import os       #importing os to set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable

### Cargo los parquet

In [3]:
#raw_data_act = ks.read_parquet('activityObfmod5.parquet',columns=['user_id','pediodo'])
#raw_data_sb = ks.read_parquet('sbObfmod5.parquet')

## Mod 5
raw_data_act = pd.read_parquet('activityObfmod5.parquet')
raw_data_sb = pd.read_parquet('sbObfmod5.parquet')

#raw_data_act = pd.read_parquet('activityObf.parquet')
#raw_data_sb = pd.read_parquet('sbObf.parquet')

### Funciones

In [2]:
def hist_actividad(df, feature_name, year, histogram = True):
    '''Si histogram == False, devuelve un dataframe con los períodos de actividad y user_id de determinado año'''
    #aux = df[(df[feature_name] >= datetime.date(year,1,1)) & (df[feature_name] < datetime.date(year+1,1,1))]  
    aux = df[(df[feature_name] >= '{}-1-1'.format(year) ) & (df[feature_name] < '{}-1-1'.format(year+1) )] 
    
    if histogram == True:
        plt.hist(aux[feature_name]);        
        plt.title(year, fontsize = 20)
    
    else:
        return aux

def get_month_diff(a, b):
      
      return 12*(a.year - b.year) + (a.month - b.month)       
      
def se_fuga(df):
    
      if (df['user_id'] == df['next_user_id']) and ( get_month_diff(df['next_periodo'], df['periodo']) >= 4) :
          return 1
      #elif (df['user_id'] != df['next_user_id']) and ( get_month_diff(max_date, df['periodo']) >= 4) :
      #    return 0
      else:
          return 0
    
def define_estados(df_aux):
    
      df2 = df_aux.copy()
      
      df2['next_user_id'] = np.array(df2['user_id'].shift(-1))
      df2['next_periodo'] = np.array(df2['periodo'].shift(-1))
      
      return df2.apply(se_fuga, axis=1)
    
    

### Análisis de datos

In [6]:
df_act_sorted=raw_data_act.sort_values(by=['user_id','periodo']).reset_index(drop=True) # Ordeno el dataframe de Activity
df_act_wo_duplicates = df_act_sorted.drop_duplicates(keep = 'last').reset_index(drop=True) # Saco las filas repetidas

#max_date = df_act_wo_duplicates['periodo'].max() # Fecha más reciente registrada

MemoryError: 

##### Número de usuarios:  470794

In [6]:
start_time = time.clock()

states = define_estados(df_act_wo_duplicates);
print((time.clock() - start_time)/60, "minutos")

/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


7.478314616666667 minutos


/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [ ]:
df_act_wo_duplicates[(df_act_wo_duplicates['Estado']==1) & (df_act_wo_duplicates['user_id'] == df_act_wo_duplicates['user_id'].shift(-1)) ]

In [7]:
df_act_wo_duplicates['Estado'] = states
df_act_wo_duplicates.to_csv(r'df_act_wo_duplicates.csv')


In [4]:
df_sb_sorted = raw_data_sb.sort_values(by=['user_id','periodo']).reset_index(drop=True) # Ordeno el dataframe sb
df_sb_wo_duplicates = df_sb_sorted.drop_duplicates(keep = 'last').reset_index(drop=True) # Quito filas duplicadas


In [5]:
df_sb_wo_duplicates.to_csv(r'df_sb_wo_duplicates.csv')

### CARGAR DATOS act Y sb SIN DUPLICADOS Y ORDENADOS


In [3]:
## PANDAS
df_sb_wo_duplicates = pd.read_csv('df_sb_wo_duplicates.csv')
df_act_wo_duplicates  = pd.read_csv('df_act_wo_duplicates.csv')

df_sb_wo_duplicates = df_sb_wo_duplicates.drop(columns=['Unnamed: 0'])
df_act_wo_duplicates = df_act_wo_duplicates.drop(columns=['Unnamed: 0'])

## KOALAS
# df_sb_wo_duplicates = ks.read_csv('df_sb_wo_duplicates.csv')
# df_act_wo_duplicates  = ks.read_csv('df_act_wo_duplicates.csv')

# df_sb_wo_duplicates = df_sb_wo_duplicates.drop('Unnamed: 0')
# df_act_wo_duplicates = df_act_wo_duplicates.drop('Unnamed: 0')


In [9]:
def cuenta_actividades(users):
    ''' devuelve cuántos períodos de actividad hay por user_id, 
    y cuántas filas de datos hay para ese user en el otro data set. Haciendo un vistazo rápido, 
    se ve que si el número de actividades por usuario es mayor que su cantidad de datos demográficos, 
    entonces hay datos que faltan. '''
    
    Num_actividades = np.zeros(len(users))
    Num_datos_demog = np.zeros(len(users))
    
    for i, userid in enumerate(users):

      Num_actividades[i] = len(df_act_wo_duplicates[df_act_wo_duplicates['user_id']== userid])
      Num_datos_demog[i] = len(df_sb_wo_duplicates[df_sb_wo_duplicates['user_id'] == userid])

    
    df_users = pd.DataFrame()
    df_users['User_id'] = users
    df_users['Cantidad_actividades'] = Num_actividades
    df_users['Cantidad_filas_datos_demog'] = Num_datos_demog
    
    print(np.sum(df_users['Cantidad_filas_datos_demog']))
    return df_users
  

def merge_act_sb(df_act, df_sb):
  
  index_sb = []; estado_act = [];
   
  for i in range(df_act.shape[0]):
    
      if len(df_sb[(df_sb['periodo'] == df_act['periodo'][i]) & (df_sb['user_id'] == df_act['user_id'][i]) ]) > 0:
          
          index_aux = df_sb[(df_sb['periodo'] == df_act['periodo'][i]) & (df_sb['user_id'] == df_act['user_id'][i])].index[0]
          index_sb.append(index_aux)
          estado_act.append(df_act['Estado'][i])
  
  df_sb_act = df_sb.iloc[index_sb]
  df_sb_act['estado'] = estado_act
  df_sb_act.reset_index(drop=True)
  
  return df_sb_act
    

def generar_target_activity(df_act, df_sb):
    
    ''' Genera una columna con los estados (activo=0, se fuga = 1) para la tabla con datos demog.'''
    
    len_sb = df_sb.shape[0]
    estados = np.zeros(len_sb)
    
    df_act_fugados = df_act[df_act['Estado'] == 1]
    
    for i in range(df_act_fugados.shape[0]):
        
        if df_sb[ df_sb['user_id']==df_act_fugados['user_id'][i] & df_sb['periodo']==df_act_fugados['periodo'][i] ].shape[0] == 1:
            
            index_sb_fugado = df_sb[(df_sb['user_id'] == df_act_fugados['user_id'][i]) & (df_sb['periodo'] == df_act_fugados['periodo'][i])].index[0]           
            estados[index_sb_fugado] = 1
        
        
    return estados
  
    
def chequea_existen_datos_demog(df_act, df_sb):
    '''Chequeda para cada fila de la tabla de actividades, si tiene definido datos demog 
    (compara con la tabla de user_id,período,Estado)'''
    user_id_aux = []
    periodo_aux = []
    
    for j in range(df_act.shape[0]):
        
        if len(df_sb[(df_sb['user_id']==df_act['user_id'][j]) & 
                    (df_sb['periodo']==df_act['periodo'][j])]) == 0:
            
            user_id_aux.append(df_act['user_id'][j])
            periodo_aux.append(df_act['periodo'][j])
            
    print("Existen {} pares de user_id y periodo sin datos demográficos definidos \n".format(len(user_id_aux)))
     
    
    return user_id_aux, periodo_aux
            
                
                

In [10]:
start_time = time.clock()

user_id_noact, periodo_noact = chequea_existen_datos_demog(df_act_wo_duplicates.head(1000), df_sb_wo_duplicates.head(1000))

print((time.clock() - start_time)/60, "minutos.")

/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Existen 682 pares de user_id y periodo sin datos demográficos definidos 

0.01422629999999998 minutos.


/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


In [7]:
j = 10

df_sb_wo_duplicates[(df_sb_wo_duplicates['user_id']==user_id_noact[j]) &
                     (df_sb_wo_duplicates['periodo']==periodo_noact[j]) ] 

,fecha_proceso,R04PERREF,periodo,user_id,total_deudores_informados,deuda_vigente,deuda_directa_morosa90,deuda_directa_vencida,deuda_directa_mora180,deuda_indirecta_mora180,...,deuda_indirecta_vigente,deuda_indirecta_vencida,deuda_directa_comercial,deuda_directa_cred_consumo,deuda_directa_hipotecaria,deuda_directa_comercial_ext,deuda_indirecta_comercial_ext,deuda_directa_leasing,deuda_morosa_leasing,monto_lineas_cred_disp


In [8]:
df_act_wo_duplicates[(df_act_wo_duplicates['user_id']==user_id_noact[j]) &
                     (df_act_wo_duplicates['periodo']==periodo_noact[j]) ] 

,user_id,periodo,Estado
23,100000586751768816,2017-01-01,0


In [ ]:
estados = generar_target_activity(df_act_wo_duplicates, df_sb_wo_duplicates)

sum(estados)


In [5]:
users_array = df_act_wo_duplicates['user_id'].unique()  # array con los id usuarios
  
cantidad_act_sb=cuenta_actividades(users_array)
cantidad_act_sb.head(10)

KeyboardInterrupt: 

# Resources
##Apache Spark
Framework para procesamiento distribuido.
doc: https://spark.apache.org/

##Koalas

Una libreria del estilo Pandas  que esa spark por atras para el manejo de datos.
https://koalas.readthedocs.io/en/latest/

## scikit-learn
Una libreria para machine learning

https://scikit-learn.org/stable/

##cursos copados

los cursos estos son de gran ayuda en general, **no es para que los veas ahora**, son lindos recursos para meterse de lleno en el mundo del data scientist dev.

###ML

https://www.coursera.org/courses?query=machine+learning+andrew+ng

https://www.coursera.org/specializations/deep-learning

https://www.udemy.com/machinelearning-es/

###DS

https://www.coursera.org/specializations/data-science-python

###Scala

https://www.coursera.org/specializations/scala

https://www.udemy.com/the-ultimate-hands-on-hadoop-tame-your-big-data/




Libros hay millones si queres despues te paso un repo donde tengo una cantidad infernal, dependiendo el tema.



